# Time Series Analysis using FbProphet

In [ ]:
# importing recquired packages
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/kaggle/input/microsoft-stock-time-series-analysis/Microsoft_Stock.csv')
df.head()

In [ ]:
# printing basic information about the dataset
print(df.info())
print(df.describe())

In [ ]:
# plotting the graph
df.plot(x='Date',y='Close',figsize=(16,7))

### BoxCox Transformation

The Box-Cox transformation transforms our data so that it closely resembles a normal distribution.
In many statistical techniques, we assume that the errors are normally distributed. This assumption allows us to construct confidence intervals and conduct hypothesis tests. By transforming your target variable, we can (hopefully) normalize our errors (if they are not already normal).
Additionally, transforming our variables can improve the predictive power of our models because transformations can cut away white noise.

In [ ]:
from statsmodels.base.transform import BoxCox
bc=BoxCox()

In [ ]:
df['Close'], lmbda=bc.transform_boxcox(df['Close'])

### FbProphet model implementation

Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

In [ ]:
# We need to change our column names as given below as that is how FbProphet recognizes the variables.
data=df[["Date","Close"]]
data.columns=["ds","y"]

In [ ]:
model_params={
    "daily_seasonality":False,
    "weekly_seasonality": False,
    "yearly_seasonality": True,
    "seasonality_mode": "multiplicative",
    "growth": "logistic"
}

In [ ]:
# importing fbprophet
from fbprophet import Prophet
model=Prophet(**model_params)

data["cap"]=data['y'].max() + data['y'].std()*0.05

model.fit(data)
future=model.make_future_dataframe(periods=365)
future["cap"]=data["cap"].max()
forecast=model.predict(future)

In [ ]:
model.plot_components(forecast)
model.plot(forecast)

Here, the black dots are the original values and the blue line is the predicted values. We can also see trends and other features.

Here we are doing the same thing as above, plus adding monthly and quaterly seasonality. Additionaly,we are also considering the holidays (Like Christmas) as important values. Stock prices often fluctuate differently when there are holidays.

In [ ]:
model=Prophet(**model_params)

model.add_seasonality(name="monthly",period=30,fourier_order=10)
model.add_seasonality(name="quaterly",period=92.95,fourier_order=10)
model.add_country_holidays("US")

model.fit(data)

future=model.make_future_dataframe(periods=365)
future["cap"]=data["cap"].max()
forecast=model.predict(future)

In [ ]:
model.plot_components(forecast);
model.plot(forecast);

We can clearly see the fluctuation around December (Christmas) in the holidays graph.

### Cross Validation

In [ ]:
import itertools
import numpy as np
from fbprophet.diagnostics import cross_validation,performance_metrics

param_grid={
    "daily_seasonality":[False],
    "weekly_seasonality": [False],
    "yearly_seasonality": [True],
    "growth": ["logistic"],
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0]

}

all_params=[
            dict(zip(param_grid.keys(),v))
            for v in itertools.product(*param_grid.values())
]
print(all_params)


Calculating the best parameters

In [ ]:
rmses=list()

for params in all_params:
  m=Prophet(**params)
  m=m.add_seasonality(name="monthly",period=30,fourier_order=5)
  m=m.add_seasonality(name="quaterly",period=92.25,fourier_order=10)
  m.add_country_holidays(country_name="US")
  m.fit(data)
  df_cv=cross_validation(m,initial="730 days",period="365 days", horizon="365 days",parallel="processes")
  df_p=performance_metrics(df_cv,rolling_window=1)
  rmses.append(df_p['rmse'].values[0])

  best_params=all_params[np.argmin(rmses)]
  print("\nThe best params are:",best_params)

Once again we are doing the same procedure, except this time using the best parameters available

In [ ]:
best_model=Prophet(**best_params)
best_model=best_model.add_seasonality(name="monthly",period=30,fourier_order=5)
best_model=best_model.add_seasonality(name="quaterl",period=92.25,fourier_order=10)
best_model.add_country_holidays(country_name="US")
best_model.fit(data)
future=best_model.make_future_dataframe(periods=365,freq='D')
future["cap"]=data["cap"].max()
forecast=best_model.predict(future)

In [ ]:
best_model.plot_components(forecast)
best_model.plot(forecast)

In [ ]:
# We can look at the info and columns of forecast and we can see all the holidays listed in it
forecast.info()

We need to untransform the BoxCox transformation to get it back to it's original state

In [ ]:
forecast['yhat']=bc.untransform_boxcox(x=forecast['yhat'],lmbda=lmbda)
forecast['yhat_lower']=bc.untransform_boxcox(x=forecast['yhat_lower'],lmbda=lmbda)
forecast['yhat_upper']=bc.untransform_boxcox(x=forecast['yhat_upper'],lmbda=lmbda)
forecast.plot(x='ds',y=['yhat_lower','yhat','yhat_upper'])